# HemeThrombKG/HemeKG 2.0 intracellular protein analysis


This notebook generates subgraphs of extracellular nodes in the HemeThrombKG/hemeKG 2.0, pathways and their merged representations.

In [1]:
import getpass
import json
import pandas as pd
import sys
import time
import urllib.request

import pybel

from network_utils import get_bel_graph, get_node_paths, get_nodes_by_region

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tqdm/autonotebook/__init__.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  warn("Using `tqdm.autonotebook.tqdm` in notebook mode."


In [2]:
sys.version

'3.8.6 (v3.8.6:db455296be, Sep 23 2020, 13:31:39) \n[Clang 6.0 (clang-600.0.57)]'

In [3]:
getpass.getuser()

'sarahbeenie'

In [4]:
time.asctime()

'Sun Sep 18 18:03:06 2022'

In [5]:
pybel.get_version()

'0.15.5'

### Load HemeThrombKG

In [6]:
review_kg = pybel.from_bel_script('../bel_files/hemeThrombKG.bel')

### Load HemeKG

In [7]:
with urllib.request.urlopen('https://raw.githubusercontent.com/hemekg/hemekg/master/hemekg/_cache.bel.nodelink.json') as url:
    data = json.loads(url.read().decode())
    
    heme_kg = pybel.from_nodelink(data)

### Load pathways

In [8]:
coagulation_pathway = get_bel_graph('coagulation_pathways')
intrinsic_pathway = get_bel_graph('intrinsic_pathway_reactome')
common_pathway = get_bel_graph('common_pathway_reactome')
kegg_pathway = get_bel_graph('platelet_activation_kegg')
plug_pathway = get_bel_graph('plug_formation_reactome')

### Load intracellular proteins


In [9]:
# HemeThrombKG intracellular proteins
reviewkg_proteins = ['SELE', 'GGT1', 'BCL2', 'SELP', 'MAPK1', 'ITGA3', 'Spectrin', 'Protein Kinase C', 'NLRP3', 'AKT1', 'CLEC1B', 'ITGA2B', 'NFKB1', 'THBD', 'ICAM1', 'VCAM1', 'Actins', 'TLR4', 'Protein 4.1', 'GP6', 'NOX', 'MLKL', 'HMOX1 protein, human', 'SYK', 'BCL2L1', 'PLCG2', 'BAD', 'hemoglobin']

# Load database intracellular proteins
ic_proteins_df = pd.read_csv('../node_type_files/intracellular_proteins.tsv', sep='\t', header=None)
db_proteins = ic_proteins_df[0].to_list()


### Get paths between intracellular proteins

In [10]:
# reviewKG
review_ic_proteins = get_nodes_by_region(review_kg, reviewkg_proteins)
review_nodes = get_node_paths(review_kg, review_ic_proteins)

# coagulation pathways
coagulation_ic_proteins = get_nodes_by_region(coagulation_pathway, db_proteins)
coagulation_nodes = get_node_paths(coagulation_pathway, coagulation_ic_proteins)

# Reactome intrinsic pathway
intrinsic_ic_proteins = get_nodes_by_region(intrinsic_pathway, db_proteins)
intrinsic_nodes = get_node_paths(intrinsic_pathway, intrinsic_ic_proteins)

# Reactome common pathway
common_ic_proteins = get_nodes_by_region(common_pathway, db_proteins)
common_nodes = get_node_paths(common_pathway, common_ic_proteins)

# KEGG platelet activation pathway
kegg_ic_proteins = get_nodes_by_region(kegg_pathway, db_proteins)
kegg_nodes = get_node_paths(kegg_pathway, kegg_ic_proteins)

# Reactome plug formation pathway
plug_ic_proteins = get_nodes_by_region(plug_pathway, db_proteins)
plug_nodes = get_node_paths(plug_pathway, plug_ic_proteins)

### Merge reviewKG + pathways

In [11]:
# Merge reviewKG and coagulation graph
review_coagulation_graph = pybel.union([review_kg, coagulation_pathway])
merged_coagulation_nodes = coagulation_nodes + review_nodes

# Merge reviewKG and intrinsic pathway graph
review_intrinsic_graph = pybel.union([review_kg, intrinsic_pathway])
merged_intrinsic_nodes = intrinsic_nodes + review_nodes

# Merge reviewKG and common pathway graph
review_common_graph = pybel.union([review_kg, common_pathway])
merged_common_nodes = common_nodes + review_nodes

# Merge reviewKG and kegg pathway graph
review_kegg_graph = pybel.union([review_kg, kegg_pathway])
merged_kegg_nodes = kegg_nodes + review_nodes

# Merge reviewKG and plug pathway graph
review_plug_graph = pybel.union([review_kg, plug_pathway])
merged_plug_nodes = plug_nodes + review_nodes

###  HemeThrombKG intracellular nodes

In [12]:
pybel.to_jupyter(review_kg.subgraph(review_nodes))

<IPython.core.display.Javascript object>

### Merged coagulation pathways - HemeThrombKG intracellular nodes

In [13]:
pybel.to_jupyter(review_coagulation_graph.subgraph(merged_coagulation_nodes))

<IPython.core.display.Javascript object>

### Common pathway - HemeThrombKG intracellular nodes

In [14]:
pybel.to_jupyter(review_common_graph.subgraph(merged_common_nodes))

<IPython.core.display.Javascript object>

### KEGG pathway - HemeThrombKG intracellular nodes

In [15]:
pybel.to_jupyter(review_kegg_graph.subgraph(merged_kegg_nodes))

<IPython.core.display.Javascript object>

### Plug pathway - HemeThrombKG intracellular nodes

In [16]:
pybel.to_jupyter(review_plug_graph.subgraph(merged_plug_nodes))

<IPython.core.display.Javascript object>